<a href="https://colab.research.google.com/github/wildlifeai/spyfish_analysis/blob/main/concat_buv_doc_oct_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
!pip install boto3
# base imports
import os
import pandas as pd
import getpass
import gdown
import zipfile
import boto3
import logging
import sys
import urllib
import subprocess
import ipywidgets as widgets
from tqdm import tqdm
from pathlib import Path
import datetime


# widget imports
from IPython.display import display
from ipywidgets import Layout
import ipywidgets as widgets

# Logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

def aws_credentials():
    # Save your access key for the s3 bucket.
    aws_access_key_id = getpass.getpass("Enter the key id for the aws server")
    aws_secret_access_key = getpass.getpass(
        "Enter the secret access key for the aws server"
    )

    return aws_access_key_id, aws_secret_access_key


def connect_s3(aws_access_key_id: str, aws_secret_access_key: str):
    # Connect to the s3 bucket
    client = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
    )
    return client


def get_aws_client():
    # Set aws account credentials
    aws_access_key_id, aws_secret_access_key = os.getenv("SPY_KEY"), os.getenv(
        "SPY_SECRET"
    )
    if aws_access_key_id is None or aws_secret_access_key is None:
        aws_access_key_id, aws_secret_access_key = aws_credentials()

    # Connect to S3
    client = connect_s3(aws_access_key_id, aws_secret_access_key)

    return client


def get_matching_s3_objects(
    client: boto3.client, bucket: str, prefix: str = "", suffix: str = ""
):
    """
    ## Code modified from alexwlchan (https://alexwlchan.net/2019/07/listing-s3-keys/)
    Generate objects in an S3 bucket.

    :param client: S3 client.
    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """

    paginator = client.get_paginator("list_objects_v2")

    kwargs = {"Bucket": bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix,)
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                break

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(
    client: boto3.client, bucket: str, prefix: str = "", suffix: str = ""
):
    """
    ## Code from alexwlchan (https://alexwlchan.net/2019/07/listing-s3-keys/)
    Generate the keys in an S3 bucket.

    :param client: S3 client.
    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    return a list of the matching objects
    """

    # Select the relevant bucket
    s3_keys = [
        obj["Key"] for obj in get_matching_s3_objects(client, bucket, prefix, suffix)
    ]

    return s3_keys

def download_object_from_s3(
    client: boto3.client,
    *,
    bucket: str,
    key: str,
    version_id: str = None,
    filename: str,
):
    """
    Download an object from S3 with a progress bar.

    From https://alexwlchan.net/2021/04/s3-progress-bars/
    """

    # First get the size, so we know what tqdm is counting up to.
    # Theoretically the size could change between this HeadObject and starting
    # to download the file, but this would only affect the progress bar.
    kwargs = {"Bucket": bucket, "Key": key}

    if version_id is not None:
        kwargs["VersionId"] = version_id

    object_size = client.head_object(**kwargs)["ContentLength"]

    if version_id is not None:
        ExtraArgs = {"VersionId": version_id}
    else:
        ExtraArgs = None

    with tqdm(
        total=object_size,
        unit="B",
        unit_scale=True,
        desc=filename,
        position=0,
        leave=True,
    ) as pbar:
        client.download_file(
            Bucket=bucket,
            Key=key,
            ExtraArgs=ExtraArgs,
            Filename=filename,
            Callback=lambda bytes_transferred: pbar.update(bytes_transferred),
        )

def upload_file_to_s3(client: boto3.client, *, bucket: str, key: str, filename: str):
    """
    > Upload a file to S3, and show a progress bar if the file is large enough
    :param client: The boto3 client to use
    :param bucket: The name of the bucket to upload to
    :param key: The name of the file in S3
    :param filename: The name of the file to upload
    """

    # Get the size of the file to upload
    file_size = os.stat(filename).st_size

    # Prevent issues with small files (<1MB) and tqdm
    if file_size > 1000000:
        with tqdm(
            total=file_size,
            unit="B",
            unit_scale=True,
            desc=filename,
            position=0,
            leave=True,
        ) as pbar:
            client.upload_file(
                Filename=filename,
                Bucket=bucket,
                Key=key,
                Callback=lambda bytes_transferred: pbar.update(bytes_transferred),
            )
    else:
        client.upload_file(
            Filename=filename,
            Bucket=bucket,
            Key=key,
        )

def delete_file_from_s3(client: boto3.client, *, bucket: str, key: str):
    """
    > Delete a file from S3.

    :param client: boto3.client - the client object that you created in the previous step
    :type client: boto3.client
    :param bucket: The name of the bucket that contains the object to delete
    :type bucket: str
    :param key: The name of the file
    :type key: str
    """
    client.delete_object(Bucket=bucket, Key=key)

def get_movie_extensions():
    # Specify the formats of the movies to select
    return tuple(["wmv", "mpg", "mov", "avi", "mp4", "MOV", "MP4"])

def check_movies_from_server(client):
    """
    It takes in a dataframe with movies information and a dictionary with the database information, and
    returns two dataframes: one with the movies that are missing from the server, and one with the
    movies that are missing from the csv

    :param db_info_dict: a dictionary with the following keys:
    :param project: the project object
    """
    # Download movies csv
    download_object_from_s3(
            client,
            bucket="marine-buv",
            key="init_db_doc_buv/movies_buv_doc.csv",
            filename="movies_buv_doc.csv"
        )

    # Load the csv with movies information
    movies_df = pd.read_csv("movies_buv_doc.csv")

    # Get a dataframe of all movies from AWS
    movies_s3_pd = get_matching_s3_keys(
        client = client,
        bucket = "marine-buv",
        suffix=get_movie_extensions(),
    )

    # Calling DataFrame constructor on list
    movies_s3_pd = pd.DataFrame(movies_s3_pd, columns =['Key'])
    # Specify the key of the movies (path in S3 of the object)
    movies_s3_pd["filename"] = movies_s3_pd.Key.str.split("/").str[-1]

    # Create a column with the deployment folder of each movie
    movies_s3_pd["deployment_folder"] = (
        movies_s3_pd.Key.str.split("/").str[:2].str.join("/")
    )

    # Missing info for files in the "buv-zooniverse-uploads"
    missing_info = movies_df.merge(
        movies_s3_pd, on=["filename"], how="outer", indicator=True
    )

    # Find out files missing from the Server
    missing_from_server = missing_info[missing_info["_merge"] == "left_only"]

    logging.info(f"There are {len(missing_from_server.index)} movies missing")

    # Find out files missing from the csv
    missing_from_csv = missing_info[missing_info["_merge"] == "right_only"].reset_index(
        drop=True
    )

    logging.info(
        f"There are {len(missing_from_csv.index)} movies missing from movies.csv"
    )

    return missing_from_server, missing_from_csv

def select_deployment(missing_from_csv: pd.DataFrame):
    """
    > This function takes a dataframe of missing files and returns a widget that allows the user to
    select the deployment of interest

    :param missing_from_csv: a dataframe of the files that are in the data folder but not in the csv file
    :return: A widget object
    """
    if missing_from_csv.shape[0] > 0:
        # Widget to select the deployment of interest
        deployment_widget = widgets.SelectMultiple(
            options=sorted(missing_from_csv.deployment_folder.unique()),
            description="New deployment:",
            disabled=False,
            rows=10,
            layout=Layout(width="80%"),
            style={"description_width": "initial"},
        )
        display(deployment_widget)

        return deployment_widget

def select_eventdate():
    """
    > This function creates a date picker widget that allows the user to select a date.
    The function is called `select_eventdate()` and it returns a date picker widget.
    :return: The date widget
    """
    # Select the date
    date_widget = widgets.DatePicker(
        description="Date of deployment:",
        value=datetime.date.today(),
        disabled=False,
        layout=Layout(width="50%"),
        style={"description_width": "initial"},
    )
    display(date_widget)

    return date_widget

def concatenate_videos(movie_list, output_file):
    # Write relative paths to a temporary text file
    temp_file_path = os.path.abspath("temp_file.txt")
    with open(temp_file_path, "w") as textfile:
        for movie_i in movie_list:
            textfile.write(f"file '{movie_i}'\n")

    textfile.close()

    # Concatenate the videos
    try:
        result = subprocess.run(
            [
                "ffmpeg",
                "-f",
                "concat",
                "-safe",
                "0",
                "-i",
                temp_file_path,
                "-c",
                "copy",
                output_file,
            ],
            check=True,
            capture_output=True,
            text=True
        )

    except subprocess.CalledProcessError as e:
        print(f"Error during concatenation: {e}")
        print(e.stderr)

    # Remove the temporary text file
    os.remove(temp_file_path)


def update_new_deployments(
    deployment_selected, event_date: widgets.Widget, delete_go_pro_s3=False, test_concatenation=False
):
    for deployment_i in deployment_selected.value:
        logging.info(
            f"Starting to concatenate {deployment_i} out of {len(deployment_selected.value)} deployments selected"
        )

        # Get a dataframe of movies from the deployment
        movies_s3_pd = get_matching_s3_keys(
            client = client,
            bucket = "marine-buv",
            prefix=deployment_i,
            suffix=get_movie_extensions(),
        )

        # Calling DataFrame constructor on list
        movies_s3_pd = pd.DataFrame(movies_s3_pd, columns =['Key'])

        # Create a list of the list of movies inside the deployment selected
        movie_files_server = movies_s3_pd.Key.unique().tolist()

        if len(movie_files_server) < 2:
            logging.info(
              f"Deployment {deployment_i} will not be concatenated because it only has {movies_s3_pd.Key.unique()}"
            )
        else:
            # Concatenate the files if multiple
            logging.info(f"The files {movie_files_server} will be concatenated")

            # Start text file and list to keep track of the videos to concatenate
            movie_list = []

            for movie_i in sorted(movie_files_server):
                # Specify the temporary output of the go pro file
                movie_i_output = movie_i.split("/")[-1]

                # Download the files from the S3 bucket
                if not os.path.exists(movie_i_output):
                    download_object_from_s3(
                        client = client,
                        bucket = "marine-buv",
                        key=movie_i,
                        filename=movie_i_output,
                    )

                # Keep track of the videos to concatenate
                movie_list.append(movie_i_output)

            # Save eventdate as str
            EventDate_str = event_date.value.strftime("%d_%m_%Y")

            # Specify the name of the concatenated video
            filename = deployment_i.split("/")[-1] + "_" + EventDate_str + ".MP4"

            # Concatenate the files
            if not os.path.exists(filename):
                logging.info(f"Concatenating {filename}")

                # Concatenate the videos
                concatenate_videos(movie_list, filename)

            if test_concatenation:
                logging.info(f"{filename} concatenated but not uploaded to the S3 bucket")

            else:
                # Upload the concatenated video to the S3
                upload_file_to_s3(
                    client = client,
                    bucket = "marine-buv",
                    key=deployment_i + "/" + filename,
                    filename=filename,
                )

                logging.info(f"{filename} successfully uploaded to {deployment_i}")

                # Delete the raw videos downloaded from the S3 bucket
                for f in movie_list:
                    os.remove(f)

                # Delete the concat video
                os.remove(filename)

                if delete_go_pro_s3:
                    # Delete the movies from the S3 bucket
                    for movie_i in sorted(movie_files_server):
                        delete_file_from_s3(
                            client=client,
                            bucket="marine-buv",
                            key=movie_i,
                        )

# Connect to s3

In [ ]:
# Your acess key for the s3 bucket.
aws_access_key_id = getpass.getpass('Enter the key id for the aws server')
aws_secret_access_key = getpass.getpass('Enter the secret access key for the aws server')

In [3]:
# Connect to the s3 bucket
client = boto3.client('s3',
                      aws_access_key_id = aws_access_key_id,
                      aws_secret_access_key = aws_secret_access_key)

# Get info from go pro movies

In [ ]:
missing_from_server, missing_from_csv = check_movies_from_server(
    client
)

Select deployments recorded on the same date

In [ ]:
deployment_selected = select_deployment(missing_from_csv)

Select date of recording

In [ ]:
event_date = select_eventdate()

Concatenate the movies

In [ ]:
update_new_deployments(deployment_selected, event_date)